In [1]:
!pip install pandas==0.24.0
!pip install tqdm==4.32.2
!pip install tqdm boto3 requests regex
!pip install pytorch_pretrained_bert pytorch-nlp

  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\umar shehu\\appdata\\roaming\\python\\python37\\site-packages\\~andas\\io\\msgpack\\_packer.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Found existing installation: tqdm 4.36.1
    Uninstalling tqdm-4.36.1:
      Successfully uninstalled tqdm-4.36.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\UMARSH~1\\AppData\\Local\\Temp\\pip-uninstall-8mev_54p\\tqdm.exe'
Consider using the `--user` option or check the permissions.



  ERROR: Could not find a version that satisfies the requirement torch>=0.4.1 (from pytorch_pretrained_bert) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch>=0.4.1 (from pytorch_pretrained_bert)


In [2]:
%matplotlib inline
import sys
import itertools
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pandas as pd
tqdm.pandas()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

# Load data

In [3]:
dataset = pd.read_json('/kaggle/input/News_Category_Dataset_v2.json', lines=True)

In [4]:
dataset.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


# preprocessing

## fix categories

In [5]:
cats = dataset.groupby('category')
print(cats.size(), "\n total groups: ",cats.ngroups)

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [6]:
# merge categories which should be the same
dataset.category.replace('THE WORLDPOST', 'WORLDPOST', inplace=True)
dataset.category.replace('WORLDPOST', 'WORLD NEWS', inplace=True)
dataset.category.replace('ARTS', 'ARTS & CULTURE', inplace=True)
dataset.category.replace('CULTURE & ARTS', 'ARTS & CULTURE', inplace=True)
dataset.category.replace('PARENTS', 'PARENTING', inplace=True)
dataset.category.replace('STYLE', 'STYLE & BEAUTY', inplace=True)

## make new contents column

In [7]:
# headline plus short description
dataset['contents'] = dataset.headline + ". " + dataset.short_description

## split the data into train and test

In [8]:
train, test = train_test_split(dataset, test_size=0.2)

In [9]:
train.head()

,authors,category,date,headline,link,short_description,contents
105477,"Carol Brailey, ContributorImage Consultant and...",STYLE & BEAUTY,2014-11-28,5 Steps to Efficiently Tackle Discount Racks T...,https://www.huffingtonpost.com/entry/5-steps-t...,For some shoppers it is a thrill to sort throu...,5 Steps to Efficiently Tackle Discount Racks T...
8045,Mary Papenfuss,POLITICS,2018-01-09,Michael Wolff Calls Trump The 'Biggest Leaker'...,https://www.huffingtonpost.com/entry/trump-big...,"The president calls a ""coterie of friends and ...",Michael Wolff Calls Trump The 'Biggest Leaker'...
189012,"Maria Rodale, Contributor\nCEO and Chairman of...",STYLE & BEAUTY,2012-06-04,Shopping with a Conscience: Treasure and Bond,https://www.huffingtonpost.com/entry/shopping-...,"Through a quick Google search, I discovered th...",Shopping with a Conscience: Treasure and Bond....
49736,"Abdi Sheikh and Feisal Omar, Reuters",WORLD NEWS,2016-08-25,Al Shabaab Militants Attack Somali Beach Resta...,https://www.huffingtonpost.com/entry/somali-be...,A gunfight is reportedly ongoing.,Al Shabaab Militants Attack Somali Beach Resta...
43952,Sam Stein and Sam Levine,POLITICS,2016-10-29,FBI Director James Comey Knew The Risks With H...,https://www.huffingtonpost.com/entry/donald-tr...,Trump's already mischaracterizing the investig...,FBI Director James Comey Knew The Risks With H...


In [10]:
train_contents, train_cats = train['contents'] , train['category'] # select the test and training data
test_contents, test_cats = test['contents'] , test['category']
len(train_contents), len(train_cats), len(test_contents), len(test_cats)

(160682, 160682, 40171, 40171)

## Tokenize

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # initialise the BERT tokenizer

100%|██████████| 231508/231508 [00:00<00:00, 924843.95B/s]


In [12]:
MAX_LEN = 128

In [13]:
train_tokens = train_contents.progress_apply(lambda x: ['[CLS]'] + tokenizer.tokenize(x)[:MAX_LEN] + ['[SEP]']) # BERT accepts a [CLS] token as the start of a sentence, so for each dataset add this and clip it at max 511 lenth. [SEP] should be at the end?
test_tokens = test_contents.progress_apply(lambda x: ['[CLS]'] + tokenizer.tokenize(x)[:MAX_LEN] + ['[SEP]'])

100%|██████████| 40171/40171 [00:27<00:00, 1457.75it/s]


In [14]:
train_tokens.iloc[0]

['[CLS]',
 '5',
 'steps',
 'to',
 'efficiently',
 'tackle',
 'discount',
 'racks',
 'this',
 'black',
 'friday',
 '.',
 'for',
 'some',
 'shop',
 '##pers',
 'it',
 'is',
 'a',
 'thrill',
 'to',
 'sort',
 'through',
 'all',
 'the',
 'racks',
 'and',
 'find',
 'hidden',
 'gems',
 ',',
 'while',
 'for',
 'others',
 'shopping',
 'the',
 'discount',
 'racks',
 'can',
 'be',
 'overwhelming',
 'and',
 'time',
 '-',
 'consuming',
 ',',
 'particularly',
 'if',
 'they',
 'do',
 'not',
 'like',
 'shopping',
 'or',
 'if',
 'they',
 'have',
 'little',
 'time',
 'on',
 'their',
 'hands',
 '.',
 'but',
 'not',
 'to',
 'worry',
 '!',
 'there',
 'is',
 'a',
 'way',
 'to',
 'minimize',
 'your',
 'shopping',
 'time',
 '.',
 '[SEP]']

In [15]:
train_tokens_ids = train_tokens.progress_apply(tokenizer.convert_tokens_to_ids) # convert the tokens to ids
test_tokens_ids = test_tokens.progress_apply(tokenizer.convert_tokens_to_ids)

100%|██████████| 40171/40171 [00:00<00:00, 70270.37it/s]


In [16]:
train_tokens_ids

105477    [101, 1019, 4084, 2000, 18228, 11147, 19575, 2...
8045      [101, 2745, 23498, 4455, 8398, 1996, 1005, 522...
189012    [101, 6023, 2007, 1037, 13454, 1024, 8813, 199...
49736     [101, 2632, 21146, 3676, 7875, 17671, 2886, 16...
43952     [101, 8495, 2472, 2508, 2272, 2100, 2354, 1996...
29346     [101, 4922, 8665, 1024, 5977, 17688, 2099, 884...
24726     [101, 4905, 2236, 5076, 6521, 7283, 2056, 2002...
80441     [101, 13225, 2115, 1005, 3400, 1005, 26369, 20...
174088    [101, 5616, 2962, 5030, 2125, 24108, 3372, 102...
81007     [101, 1996, 2028, 3696, 1997, 12520, 2057, 203...
112958    [101, 10733, 12570, 13999, 10733, 2007, 11611,...
162813    [101, 17974, 1997, 1996, 2154, 1024, 3803, 333...
21744     [101, 2058, 2028, 2353, 1997, 1057, 1012, 1055...
185823    [101, 2242, 2000, 8439, 1024, 2459, 1997, 1996...
154300    [101, 2437, 1996, 2553, 2005, 5306, 1037, 2919...
169259    [101, 10556, 2571, 11772, 17974, 1024, 8670, 3...
138078    [101, 4977, 2015, 2131, 12403,

In [17]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=MAX_LEN, truncating="post", padding="post", dtype="int") # pad sequences
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=MAX_LEN, truncating="post", padding="post", dtype="int")

In [18]:
train_tokens_ids # they're now numpy arrays

array([[  101,  1019,  4084, ...,     0,     0,     0],
       [  101,  2745, 23498, ...,     0,     0,     0],
       [  101,  6023,  2007, ...,     0,     0,     0],
       ...,
       [  101,  2122, 12043, ...,     0,     0,     0],
       [  101,  1037,  5507, ...,     0,     0,     0],
       [  101,  2403,  5030, ...,     0,     0,     0]])

In [19]:
train_tokens_ids.shape, test_tokens_ids.shape

((160682, 128), (40171, 128))

## now convert the labels to numbers

In [20]:
train_y = train_cats.astype('category').cat.codes #  convert each categorical label into a number to label the category
test_y = test_cats.astype('category').cat.codes

In [21]:
train_y = train_y.astype('int64').values
test_y = test_y.astype('int64').values

In [22]:
#  now convert these to one hot encoded!
#no_labels = len(train_cats.astype('category').cat.categories)
#train_y = np.eye(no_labels)[train_y] # creates onehot array
#test_y = np.eye(no_labels)[test_y]

In [23]:
train_y.shape, test_y.shape

((160682,), (40171,))

## masks

In [24]:
train_masks = (train_tokens_ids > 0).astype('float')
train_masks = train_masks.tolist()

test_masks = (test_tokens_ids > 0).astype('float')
test_masks = test_masks.tolist()

In [25]:
train_masks

[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0

# run baseline

In [26]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), SGDClassifier(loss='log', verbose=2, n_jobs=-1, max_iter=10, learning_rate='optimal', n_iter_no_change=5, validation_fraction=0.1, alpha=0.001)) # use sgd as we have a large model. log loss makes it logistic regression

In [27]:
baseline_model = baseline_model.fit(train_contents, train_cats)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


-- Epoch 1-- Epoch 1

Norm: 5.58, NNZs: 4729598, Bias: -1.355183, T: 160682, Avg. loss: 0.092620
Total training time: 0.48 seconds.
-- Epoch 2
Norm: 5.53, NNZs: 4729598, Bias: -1.288718, T: 160682, Avg. loss: 0.103739
Total training time: 0.54 seconds.
-- Epoch 2
Norm: 5.44, NNZs: 4729598, Bias: -1.459334, T: 321364, Avg. loss: 0.079954
Total training time: 0.91 seconds.
-- Epoch 3
Norm: 5.38, NNZs: 4729598, Bias: -1.397894, T: 321364, Avg. loss: 0.089307
Total training time: 0.97 seconds.
-- Epoch 3
Norm: 5.37, NNZs: 4729598, Bias: -1.516903, T: 482046, Avg. loss: 0.078225
Total training time: 1.36 seconds.
-- Epoch 4
Norm: 5.31, NNZs: 4729598, Bias: -1.457864, T: 482046, Avg. loss: 0.087350
Total training time: 1.44 seconds.
-- Epoch 4
Norm: 5.32, NNZs: 4729598, Bias: -1.556231, T: 642728, Avg. loss: 0.077372
Total training time: 1.78 seconds.
-- Epoch 5
Norm: 5.27, NNZs: 4729598, Bias: -1.499228, T: 642728, Avg. loss: 0.086349
Total training time: 1.86 seconds.
-- Epoch 5
Norm: 5.30

[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:  1.0min finished


In [28]:
import pickle
pkl_filename = "../working/news_logistic_bigsgd_higherreg.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(baseline_model, file)

In [29]:
import pickle
pkl_filename = "../working/news_logistic_bigsgd_higherreg.pkl"
pickle.load(open(pkl_filename, "rb"))

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, voc...
                ('sgdclassifier',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='log', max_iter=10,
 

In [30]:
baseline_predicted = baseline_model.predict(test_contents) # predict the categories for the test set

In [31]:
print(classification_report(test_cats, baseline_predicted)) # report the scores

                precision    recall  f1-score   support

ARTS & CULTURE       0.67      0.23      0.34       799
  BLACK VOICES       0.54      0.20      0.29       876
      BUSINESS       0.53      0.26      0.35      1233
       COLLEGE       0.47      0.12      0.19       233
        COMEDY       0.64      0.22      0.33      1052
         CRIME       0.55      0.41      0.47       708
       DIVORCE       0.92      0.51      0.66       738
     EDUCATION       0.41      0.09      0.15       202
 ENTERTAINMENT       0.44      0.75      0.55      3098
   ENVIRONMENT       0.96      0.10      0.19       249
         FIFTY       0.12      0.01      0.01       279
  FOOD & DRINK       0.59      0.63      0.61      1249
     GOOD NEWS       0.42      0.02      0.03       316
         GREEN       0.44      0.11      0.17       528
HEALTHY LIVING       0.53      0.11      0.19      1346
 HOME & LIVING       0.77      0.52      0.62       813
        IMPACT       0.42      0.09      0.15  

# BERT pretrained model

## general setup

In [32]:
rn.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## load classifier

In [34]:
from pytorch_pretrained_bert import BertForSequenceClassification

In [35]:
no_labels = len(train_cats.astype('category').cat.categories)
bert_clf = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=no_labels)
bert_clf = bert_clf.cuda()

100%|██████████| 407873900/407873900 [00:15<00:00, 25963421.05B/s]


## Load model params

In [36]:
BATCH_SIZE = 64
EPOCHS = 4

In [37]:
param_optimizer = list(bert_clf.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [38]:
from pytorch_pretrained_bert import BertAdam
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=3e-5,
                     warmup=.1)

## setup data as tensors for pytorch

In [39]:
# convert to tensors
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y, dtype=torch.long)

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y, dtype=torch.long)

train_masks_tensor = torch.tensor(train_masks,  dtype=torch.long)
test_masks_tensor = torch.tensor(test_masks,  dtype=torch.long)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.169536M'

In [40]:
# convert a;; to dataloader to save memory later
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [41]:
loss_func = nn.BCEWithLogitsLoss().cuda()

In [42]:
# todo:
# split data into validation set also?
# add code for validation evalutation?
# clean up this notebook a bit.
# evaluate the classifier
# 

In [43]:
token_ids.shape, masks.shape, labels.shape

NameError: name 'token_ids' is not defined

In [44]:
losses = []
steps = []
step = 0
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader): # train for each epoch
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        optimizer.zero_grad() # clear gradients
        loss =  bert_clf(input_ids=token_ids, attention_mask=masks, labels=labels) # forward pass
        losses.append(loss.item())
        loss.backward() # backward pass
        #import ipdb
        #ipdb.set_trace()

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        
        optimizer.step() # update params and step using computed gradient
        
        clear_output(wait=True)
        
        train_loss += loss.item()
        steps.append(step)
        step += 1
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(train) / BATCH_SIZE, train_loss / (step_num + 1)))
        
    torch.save({
            'epoch': epoch_num,
            'model_state_dict': bert_clf.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, "../working/news_bert_chpt.pt")

Epoch:  4
530/2510.65625 loss: 0.33213921919773753 


In [45]:
PATH = "../working/news_bert_statedict.pt"
toch.save(bert_clf.state_dict(), PATH)

NameError: name 'toch' is not defined

In [46]:
PATH = "../working/news_bert.pt"
toch.save(bert_clf, PATH)

NameError: name 'toch' is not defined

In [47]:
bert_clf.eval() # put model in evaluation mode
bert_predicted = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks = tuple(t.to(device) for t in batch_data)
        logits_predictions =  bert_clf(input_ids=token_ids, attention_mask=masks) # forward pass
        
        logits = logits_predictions.detach().cpu().numpy() #move to cpu
        
        bert_predicted += (np.argmax(logits, axis=1)).tolist()

ValueError: too many values to unpack (expected 2)

In [48]:
print(classification_report(test_y, bert_predicted))

ValueError: Found input variables with inconsistent numbers of samples: [40171, 0]